<img src="encabezado.jpg" width="800"/>

“Perú a la carta” es una propuesta de plataforma digital que se enfoca en proporcionar a los usuarios una experiencia integral y personalizada al planificar viajes al Perú, combinando información detallada sobre vuelos y actividades turísticas. 
Nuestra plataforma busca simplificar y mejorar la toma de decisiones de los viajeros al ofrecer datos actualizados y relevantes, y a su vez, permite a la empresa ofertante de servicios personalizar itinerarios según las preferencias de sus clientes. 

## Importación de codigos IATA

Los códigos IATA, que corresponden a la Asociación Internacional de Transporte Aéreo (IATA, por sus siglas en inglés), son códigos de tres letras utilizados para identificar de manera única a los aeropuertos y a las aerolíneas en todo el mundo. Estos códigos facilitan la identificación y el procesamiento de información en la industria de la aviación.
Por ejemplo, el código IATA para el Aeropuerto Internacional de Los Ángeles es LAX
De manera similar, las aerolíneas también se identifican mediante códigos IATA de dos letras. Por ejemplo, American Airlines tiene el código AA

In [1]:
pip install amadeus

Note: you may need to restart the kernel to use updated packages.


In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as ptl
import seaborn as sns
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
import requests
import json
from amadeus import Client, ResponseError

In [18]:
import csv
import os

# Nombre del archivo CSV
nombre_archivo = "cod_iata.csv"

# Obtener el directorio de trabajo actual
directorio_actual = os.getcwd()

# Unir la ruta del directorio actual con el nombre del archivo
ruta_archivo = os.path.join(directorio_actual, nombre_archivo)

# Resto del código
datos_cod_iata = {}

with open(ruta_archivo, 'r', newline='', encoding='utf-8') as archivo_csv:
    lector_csv = csv.reader(archivo_csv)
    encabezado = next(lector_csv)

    for fila in lector_csv:
        codigo_iata = fila[0]
        otros_valores = fila[1:] if len(fila) > 1 else None
        datos_cod_iata[codigo_iata] = otros_valores[0] if otros_valores else None

ciudad_input = input("Ingresa el nombre de la ciudad: ")

codigo_iata_encontrado = datos_cod_iata.get(ciudad_input.upper(), None)

if codigo_iata_encontrado:
    print(f"Ciudad encontrada")
else:
    print(f"Ciudad no encontrada")

Ingresa el nombre de la ciudad: SANTIAGO DE CHILE
Ciudad encontrada


## Call de vuelos del API de Amadeus

<img src="amadeus_OK.jpg" width="800"/>

In [65]:
# Configuración de nuestro usuario de Amadeus
amadeus = Client(
    client_id='OvHfoJ8hyt7lzWXAAnCrxSf7dJTT9XTP',
    client_secret='ZJ6yBQd1nhdhxGgG'
)

# Buscamos las ofertas de vuelos con el API de Amadeus
ciudad_input = input("Ciudad de origen: ")
Id = input("Ingresa la fecha de ida (año-mes-día): ")
Re = input("Ingresa la fecha de retorno (año-mes-día): ")
try:
    vuelos = amadeus.shopping.flight_offers_search.get(
        originLocationCode=codigo_iata_encontrado,
        destinationLocationCode='LIM',
        departureDate=Id,
        returnDate=Re,
        currencyCode='USD',
        adults=1
    )

# Extraemos la lista de ofertas de vuelos desde la respuesta (Atributo "Vuelos")
    oferta_vuelos = vuelos.data

# Nos aseguramos que hay ofertas en el la respuesta del atributo "Vuelos"
    if 'offerItems' in oferta_vuelos[0]:
        
# Ordenamos las ofertas de vuelos por precio total de menor a mayor
        sorted_oferta_vuelos = sorted(oferta_vuelos, key=lambda x: float(x['offerItems'][0]['price']['total']))
    else:
        
# Si la estructura es diferente, ordenamos por el precio total directamente
        sorted_oferta_vuelos = sorted(oferta_vuelos, key=lambda x: float(x['price']['total']))

# Obtenemo los vuelos más económicos según la solicitud
    top_oferta_vuelos = sorted_oferta_vuelos[:5]

# Convertimos la respuesta a formato JSON
    json_vuelos = json.loads(json.dumps(top_oferta_vuelos))

# Convierte el JSON a un DataFrame de pandas
    df_vuelos = pd.json_normalize(json_vuelos)

# Exporta el DataFrame a un archivo Excel
    df_vuelos.to_excel('vuelos.xlsx', index=False)
    print('Datos exportados a vuelos.xlsx')

except:
    print("Error")


Ciudad de origen: Santiago de chile
Ingresa la fecha de ida (año-mes-día): 2024-01-20
Ingresa la fecha de retorno (año-mes-día): 2024-01-24
Datos exportados a vuelos.xlsx


In [20]:
#Visualizamos nuestra respuesta en formato JSON
json_vuelos

[{'type': 'flight-offer',
  'id': '1',
  'source': 'GDS',
  'instantTicketingRequired': False,
  'nonHomogeneous': False,
  'oneWay': False,
  'lastTicketingDate': '2024-01-16',
  'lastTicketingDateTime': '2024-01-16',
  'numberOfBookableSeats': 7,
  'itineraries': [{'duration': 'PT3H40M',
    'segments': [{'departure': {'iataCode': 'SCL',
       'terminal': '2',
       'at': '2024-01-20T08:20:00'},
      'arrival': {'iataCode': 'LIM', 'at': '2024-01-20T10:00:00'},
      'carrierCode': 'LA',
      'number': '534',
      'aircraft': {'code': '788'},
      'operating': {'carrierCode': 'LA'},
      'duration': 'PT3H40M',
      'id': '31',
      'numberOfStops': 0,
      'blacklistedInEU': False}]},
   {'duration': 'PT3H30M',
    'segments': [{'departure': {'iataCode': 'LIM',
       'at': '2024-01-25T09:00:00'},
      'arrival': {'iataCode': 'SCL',
       'terminal': '2',
       'at': '2024-01-25T14:30:00'},
      'carrierCode': 'LA',
      'number': '535',
      'aircraft': {'code': '788'

In [21]:
#Visualizamos nuestra respuesta en un Data Frame
df_vuelos

,type,id,source,instantTicketingRequired,nonHomogeneous,oneWay,lastTicketingDate,lastTicketingDateTime,numberOfBookableSeats,itineraries,validatingAirlineCodes,travelerPricings,price.currency,price.total,price.base,price.fees,price.grandTotal,price.additionalServices,pricingOptions.fareType,pricingOptions.includedCheckedBagsOnly
0,flight-offer,1,GDS,False,False,False,2024-01-16,2024-01-16,7,"[{'duration': 'PT3H40M', 'segments': [{'depart...",[LA],"[{'travelerId': '1', 'fareOption': 'STANDARD',...",USD,300.95,206.00,"[{'amount': '0.00', 'type': 'SUPPLIER'}, {'amo...",300.95,"[{'amount': '87.20', 'type': 'CHECKED_BAGS'}]",[PUBLISHED],False
1,flight-offer,2,GDS,False,False,False,2024-01-16,2024-01-16,7,"[{'duration': 'PT3H40M', 'segments': [{'depart...",[LA],"[{'travelerId': '1', 'fareOption': 'STANDARD',...",USD,300.95,206.00,"[{'amount': '0.00', 'type': 'SUPPLIER'}, {'amo...",300.95,"[{'amount': '87.20', 'type': 'CHECKED_BAGS'}]",[PUBLISHED],False
2,flight-offer,3,GDS,False,False,False,2024-01-16,2024-01-16,7,"[{'duration': 'PT3H40M', 'segments': [{'depart...",[LA],"[{'travelerId': '1', 'fareOption': 'STANDARD',...",USD,300.95,206.00,"[{'amount': '0.00', 'type': 'SUPPLIER'}, {'amo...",300.95,"[{'amount': '87.20', 'type': 'CHECKED_BAGS'}]",[PUBLISHED],False
3,flight-offer,4,GDS,False,False,False,2024-01-16,2024-01-16,7,"[{'duration': 'PT3H40M', 'segments': [{'depart...",[LA],"[{'travelerId': '1', 'fareOption': 'STANDARD',...",USD,300.95,206.00,"[{'amount': '0.00', 'type': 'SUPPLIER'}, {'amo...",300.95,"[{'amount': '87.20', 'type': 'CHECKED_BAGS'}]",[PUBLISHED],False
4,flight-offer,5,GDS,False,False,False,2024-01-16,2024-01-16,7,"[{'duration': 'PT3H55M', 'segments': [{'depart...",[LA],"[{'travelerId': '1', 'fareOption': 'STANDARD',...",USD,300.95,206.00,"[{'amount': '0.00', 'type': 'SUPPLIER'}, {'amo...",300.95,"[{'amount': '87.20', 'type': 'CHECKED_BAGS'}]",[PUBLISHED],False


<b>Extracción de datos<b>

In [73]:
#Extracción de moneda, precio y aerolinea
ID = []
Moneda = []
Precio = []
Aerolinea = []

for i in range(len(df_vuelos["id"])):   
    ID.append("Opción" + df_vuelos["id"][i])
for i in range(len(df_vuelos["price.currency"])):   
    Moneda.append(df_vuelos["price.currency"][i])
#for i in range(len(df_vuelos["price.total"])):   
#    Precio.append(df_vuelos["price.total"][i])
for i in range(len(df_vuelos["validatingAirlineCodes"])):   
    Aerolinea.append(df_vuelos["validatingAirlineCodes"][i])

for i in range(len(df_vuelos["price.currency"])):   
    Precio.append(df_vuelos["price.currency"][i] + df_vuelos["price.total"][i])
    #Precio.append(df_vuelos["price.total"][i])    
    
print(ID)
print(Moneda)
print(Precio)
print(Aerolinea)

['Opción1', 'Opción2', 'Opción3', 'Opción4', 'Opción5']
['USD', 'USD', 'USD', 'USD', 'USD']
['USD300.95', 'USD300.95', 'USD300.95', 'USD300.95', 'USD300.95']
[['LA'], ['LA'], ['LA'], ['LA'], ['LA']]


In [70]:
#Extracción de duración del tiempo de vuelo de ida y vuelo de vuelta
Duracion = []
Duracion_ida = []
Duracion_vuelta = []

# Recorremos la lista de ofertas de vuelo
for vuelo in json_vuelos:

# Recorremos los itinerarios dentro de cada oferta de vuelo
    for itinerario in vuelo['itineraries']:
        
# Accedemos a la clave 'duration' dentro de cada itinerario
        tiempo_vuelo = itinerario['duration']
        
# Agregamos la duración a la lista
        Duracion.append(tiempo_vuelo)

# Extraemos la información de los vuelos de ida y vuelos de vuelta
for i in range(0, len(Duracion),2):
    Duracion_ida.append(Duracion[i][2:])
    
for i in range(1, len(Duracion),2):
    Duracion_vuelta.append(Duracion[i][2:])
    
# Imprime la lista de duraciones
print(Duracion_ida)
print(Duracion_vuelta)

['3H40M', '3H40M', '3H40M', '3H55M', '3H55M']
['3H30M', '3H40M', '3H40M', '3H30M', '3H30M']


## Web srcaping de "Y tú que planes":

<img src="ytuqueplanes.svg" width="200"/>

In [24]:
import pandas as pd
import numpy as np
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
from urllib.request import urlopen
import unicodedata
import requests
import re
import time

In [25]:
# Definiendo header
header= {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
      'AppleWebKit/537.11 (KHTML, like Gecko) '
      'Chrome/23.0.1271.64 Safari/537.11',
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
      'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
      'Accept-Encoding': 'none',
      'Accept-Language': 'en-US,en;q=0.8',
      'Connection': 'keep-alive'}

# Recuperando página web de los destinos de ytuqueplanes
req = urllib.request.Request(url='https://www.ytuqueplanes.com/destinos', headers=header) 
page = urllib.request.urlopen(req).read()
page = str(page)
print(page)

b'\r\n\r\n\r\n<!DOCTYPE html>\r\n<html class="no-js" lang="es-PE">\r\n\r\n<head prefix="og: https://ogp.me/ns#">\r\n    <meta http-equiv="Cache-control" content="public" />\r\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\r\n    <meta charset="utf-8" />\r\n    <meta http-equiv="x-ua-compatible" content="ie=edge" />\r\n    <meta name="viewport" content="width=device-width, initial-scale=1.0" />\r\n    <base href="https://www.ytuqueplanes.com/" />\r\n    <title>Lugares tur&#237;sticos | Turismo en el Per&#250; | Y t&#250; qu&#233; planes?</title>\r\n    <link rel="icon" type="image/png" href="https://www.ytuqueplanes.com/assets/images/logo.ico" />\r\n    <meta name="description" content="Destinos Tur&#237;sticos del Per&#250;. Planifica tu pr&#243;ximo viaje y conoce los lugares para hacer Turismo que no puedes dejar de visitar." />\r\n    <meta name="keywords" lang="es" content="lugares tut&#237;sticos peru, viajes peru, destinos turisticos" />\r\n    <link r

In [26]:
# Extrayendo los destinos (regiones)
destinos = re.findall('/destinos/.*?\"', page)[1:26]

DESTINOS = []
for i in destinos:
    DESTINOS.append(i)
    
DESTINOS = [elem[:-2] for elem in DESTINOS]
print(DESTINOS) 
print(len(DESTINOS))

# Convirtiendo a urls
urls = []
for i in DESTINOS:
    urls.append("https://www.ytuqueplanes.com"+i)
    
print(urls)
print(len(urls))

['/destinos/amazonas', '/destinos/ancash', '/destinos/apurimac', '/destinos/arequipa', '/destinos/ayacucho', '/destinos/cajamarca', '/destinos/callao', '/destinos/cusco', '/destinos/huancavelica', '/destinos/huanuco', '/destinos/ica', '/destinos/junin', '/destinos/la-libertad', '/destinos/lambayeque', '/destinos/lima', '/destinos/loreto', '/destinos/madre-de-dios', '/destinos/moquegua', '/destinos/pasco', '/destinos/piura', '/destinos/puno', '/destinos/san-martin', '/destinos/tacna', '/destinos/tumbes', '/destinos/ucayali']
25
['https://www.ytuqueplanes.com/destinos/amazonas', 'https://www.ytuqueplanes.com/destinos/ancash', 'https://www.ytuqueplanes.com/destinos/apurimac', 'https://www.ytuqueplanes.com/destinos/arequipa', 'https://www.ytuqueplanes.com/destinos/ayacucho', 'https://www.ytuqueplanes.com/destinos/cajamarca', 'https://www.ytuqueplanes.com/destinos/callao', 'https://www.ytuqueplanes.com/destinos/cusco', 'https://www.ytuqueplanes.com/destinos/huancavelica', 'https://www.ytuqu

In [27]:
# Extrayendo las ofertas de cada url de los destinos
OFERTAS = []
LI = []
for url in urls:
    # Recuperando cada url
    req = urllib.request.Request(url, headers=header) 
    page = urllib.request.urlopen(req).read()
    page = str(page)
    
    # Web scraping usando regex para hallar las ofertas
    ofertas = re.findall('/ofertas/.*?\"', page)[1:]
    
    # Agregar el enlace en una lista
    for oferta in ofertas:
        OFERTAS.append("https://www.ytuqueplanes.com"+oferta)

# Seleccionando solo los enlaces
for i in range(1, len(OFERTAS),2):
    LI.append(OFERTAS[i])
    
# Eliminando las comillas del final
OFERTA = [elem[:-1] for elem in LI]
print(OFERTA)

['https://www.ytuqueplanes.com/ofertas/ano-nuevo-en-chachapoyas1036208593', 'https://www.ytuqueplanes.com/ofertas/huaraz-y-sus-montanas-396020835', 'https://www.ytuqueplanes.com/ofertas/vacaciones-en-huaraz-2024', 'https://www.ytuqueplanes.com/ofertas/huaraz-y-sus-hermosos-paisajes', 'https://www.ytuqueplanes.com/ofertas/experiencia-tuquillo-sol-mar-y-aventura', 'https://www.ytuqueplanes.com/ofertas/huaraz-en-los-andes', 'https://www.ytuqueplanes.com/ofertas/arequipa-inolvidable-1639014976c0c0d06c-a353-4339-859b-89004104bd96', 'https://www.ytuqueplanes.com/ofertas/hotel-natura-inn-arequipa', 'https://www.ytuqueplanes.com/ofertas/arequipa-y-colca-espectacular1732526652', 'https://www.ytuqueplanes.com/ofertas/promo-vacaciones-en-crismar-experience-by-xima-hotels-834720239', 'https://www.ytuqueplanes.com/ofertas/hotel-natura-inn', 'https://www.ytuqueplanes.com/ofertas/arequipa-encantadora-tu-destino-de-vacaciones', 'https://www.ytuqueplanes.com/ofertas/conozca-el-mundo-de-las-abejas', 'ht

In [28]:
# Obtener los resultados del web scraping en una base de datos
import sqlite3

# Crear la base de datos
conn = sqlite3.connect('peru.sqlite')
cur = conn.cursor()

# Crear la tabla Actividades y las columnas
cur.execute('DROP TABLE IF EXISTS Actividades')
cur.execute('CREATE TABLE Actividades (provincia TEXT, titulo TEXT, duración TEXT, costo FLOAT, descripción TEXT, alojamiento TEXT, alimentación TEXT, traslados TEXT, tours TEXT)')
conn.commit()

for i in OFERTA:
    # Realizar la solicitud HTTP y obtener el contenido de la página
    response = requests.get(i)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Por cada página, almacenar en la lista Resultados la provincia, título, duración, costo y la descripción
    Resultados = []
    Resultados.append(soup.find_all(class_='page__header__provincia')[0].text)
    Resultados.append(soup.find_all(class_='page__header__titulo')[0].text)
    Resultados.append(soup.find_all(class_='page__header__subtitulo')[0].text)
    Resultados.append(re.findall('S/\s\d.*?\.\d.*?', (soup.find_all(class_='page__header__precio')[0].text))[0][3:])
    try:
        Resultados.append(" ".join(soup.find_all(class_='content-incluye')[0].text.split()))
    except:
        Resultados.append("No hay descripción disponible")
        
    # En cada página, almacenar en la lista Resultados la información sobre el alojamiento, alimentación, traslados y tours
    try:
        alojamiento = (" ".join(soup.find_all("div", class_='item', id='alojamiento')[0].text.split()))
        alojamiento = alojamiento.replace('Alojamiento', '',1)
        if len(alojamiento) != 0:
            alojamiento = alojamiento.replace(' ', '',1)
            Resultados.append(alojamiento)
        if len(alojamiento) == 0:
            Resultados.append("No incluye alojamiento")
    except:
        print("Error")

    try:
        alimentacion = (" ".join(soup.find_all("div", class_='item', id='alimentacion')[0].text.split()))
        alimentacion = alimentacion.replace('Alimentación', '',1)
        if len(alimentacion) != 0:
            alimentacion = alimentacion.replace(' ', '',1)
            Resultados.append(alimentacion)
        if len(alimentacion) == 0:
            Resultados.append("No incluye alimentación")
    except:
        print("Error")
        
    try:
        traslados = (" ".join(soup.find_all("div", class_='item', id='transporte')[0].text.split()))
        traslados = traslados.replace('Traslados', '',1)
        if len(traslados) != 0:
            traslados = traslados.replace(' ', '',1)
            Resultados.append(traslados)
        if len(traslados) == 0:
            Resultados.append("No incluye traslados")
    except:
        print("Error")

    try:
        tours = (" ".join(soup.find_all("div", class_='item', id='tours')[0].text.split()))
        tours = tours.replace('Tours', '',1)
        if len(tours) != 0:
            tours = tours.replace(' ', '',1)
            Resultados.append(tours)
        if len(tours) == 0:
            Resultados.append("No incluye tours")
    except:
        print("Error")

    print(Resultados)
    
    # Insertar la información obtenida para cada columna en la tabla Actividades
    cur.execute('INSERT INTO Actividades (provincia, titulo, duración, costo, descripción, alojamiento, alimentación, traslados, tours) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)', 
        (Resultados))
    conn.commit()

['Amazonas', 'Año Nuevo en Chachapoyas', '04 Días / 03 Noches', '775.0', 'Dia 01: Recojo del aeropuerto o terminal de bus Tour a los cañones del Sonche y Huancaurco, traslado al hotel, tarde libre Dia 02 Tour Kuelap + Teleféricos 10:00 am recojo del hotel, traslado al teleférico 10 minutos en carro, paseo en el teleférico, caminata de 40 minutos hacia la fortaleza de Kuelap, visita guiada y traslado al restaurant para almorzar, retorno al hotel. Día 03 Tour Catarata de Gocta 08.30 am Recojo del hotel traslado a la comunidad de Cocachimba , caminata de 5 horas ida y vuelta, almuerzo en restaurant local y retorno al hotel. Día 04 Ala hora indicada traslado del hotel al aeropuerto o terminal de bus', '03 noches de alojamiento en el hotel Utcubamba River Lodge, categoría 3 estrellas.', '3 desayunos en el hotel (americano o regional) 2 almuerzos en restaurantes locales', 'Traslado Aeropuerto o Terminal de bus/Hotel/Aeropuerto o Terminal de bus', 'Cañones del Sonche y Huancaurco, Kuelap, tel

['Arequipa', 'Hotel Natura Inn Arequipa', '02 Noches', '199.0', 'Visita a nuestra terraza - mirador Sala de reuniones exclusiva para nuestros clientes Wifi de alta velocidad Habitaciones con TV cable Baño privado con amenities y secadora de cabello. Early check in sujeto a disponibilidad', '02 noches de alojamiento en base a habitación doble o matrimonial - Hotel 3 estrellas', 'Incluye desayuno buffet para 2 personas', 'No incluye traslados', 'No incluye tours']
['Arequipa', 'Arequipa y Colca espectacular', '04 Días / 03 Noches', '620.0', 'DIA 1 TRASLADO AEROPUERTO / HOTEL ELEGIDO. AREQUIPA Y LA CAMPIÑA AREQUIPEÑA. 14:00 hrs El primer punto será el Distrito de Yanahuara donde podremos tomarnos fotos en los famosos Portales, luego nos dirigimos al Mirador de Carmen Alto,Tienda Incalpaca, Mansión del Fundador y el Molino de Sabandia. Noche de alojamiento en Arequipa. Día 02: AREQUIPA COLCA 7:30 a 8am La movilidad nos recoje e iniciamos nuestro viaje, nos detendremos en el mirador de los 

['Ayacucho', 'Altipacha Ayacucho Hotel', '02 Noches', '150.0', 'No hay descripción disponible', '2 noches en Altipacha Ayacucho Hotel, 3 estrellas superior categorizado', 'Incluye servicio desayuno buffet altipacha de 6:30 am a 9:30 am', 'No incluye traslados', 'No incluye tours']
['Cajamarca', 'Vacaciones en Cajamarca', '03 Noches', '150.0', 'A 2 cuadras de la plaza de armas Wifi gratuito Early Check in y Late Check Out con costo adicional Habitaciones confortables con TV smart con cable Baño privado Service Room Mates y agua caliente en todos los nivel 24hr', '03 noches de alojamiento en Hotel Los Reyes', 'Los desayunos se sirven de 7:00 a.m. a 10:00 a.m. Se ofrecen son continental, americano o buffet.', 'No incluye traslados', 'No incluye tours']
['Cajamarca', 'Date un gusto en Cajamarca', '03 Días / 02 Noches', '527.0', 'Día 01: CAJAMARCA Recepción y traslado al hotel. Pernocte en Cajamarca. Día 02: EXCURSIÓN A CUMBEMAYO Desayuno. 09:30 AM Recojo de su hotel y salida al tour Cumbem

['Cusco', 'Tour Valle Sagrado VIP', 'Full Day (Día completo)', '189.0', 'El tour Valle Sagrado VIP Económico es uno de los servicios más recomendados en Cusco. Comienza con el recojo del turista en la puerta de su hotel (7 am aproximadamente). El primer destino turístico a visitar es Chinchero, su sitio arqueológico y el centro textil. Luego se visitan los andenes circulares de Moray y, a poca distancia, las Salineras de Maras. Por la tarde se tomará un almuerzo buffet en Urubamba, el mejor buffet del Valle Sagrado. Después toca visitar el sitio arqueológico de Ollantaytambo (templo del sol, baños de la ñusta). En el viaje de retorno visitaremos el sitio arqueológico de Pisac. Finalmente, el transporte turístico llevará al visitante de retorno a Cusco (7 pm. aproximadamente). El tour incluye todo: recojo, transporte, entradas, guía profesional y almuerzo buffet.', 'No incluye alojamiento', 'Almuerzo buffet en restaurante Tunupa (el mejor del Valle Sagrado).', 'Minivan turístico amplio.

['Huánuco', 'Tingo María Naturaleza Pura', '02 Días / 01 Noche', '195.0', 'Día 01: Laguna de los Milagros Místico. Recojo y traslado del terminal de bus o aeropuerto al hotel reservado. Inicio del tour. Visita a la Laguna de los Milagros, paseo en bote artesanal y nado en la laguna. Utilizaremos la arcilla para la exfoliación de la piel del rostro y cuerpo. Visita al Museo Amazónico y observación del paiche “Amazónico”. Almuerzo típico de la Selva. Visitaremos la Jungla, Choza de Tarzán, árbol del deseo, árbol de la Lupuna que tiene más de 400 años de antigüedad. Retorno al hotel. Día 02: Tingo María y sus Encantos Recojo del hotel. Visita al Mirador Jacintillo, donde se tiene una vista espectacular de la unión de los ríos Huallaga y Monzón. Visita al sector Cueva de las Lechuzas en el Parque Nacional Tingo María. Además, visita al Rio Cueva o Subterráneo, Fábrica de Chocolate, Balneario Aguas Sulfurosas Medicinal. Almuerzo típico. Visitaremos la Catarata Santa Carmen y el Mirador San 

...

['Lima', 'Vacaciones de verano en el hotel Las Dalias', '01 Noche', '99.0', 'Descripción del servicio a brindar: Alojamiento. Alimentación:(desayuno y almuerzo) Desayuno Americano. Almuerzo: Los platos a elegir son los siguientes: Pato en ají. Chicharrón de pollo. Pollo a la plancha. Estacionamiento. Servicio de WIFI. Áreas recreativas ( zonas de juegos para niños, áreas verdes y sapito). Piscina (10:00 am - 7:00 pm). TV por cable.', '01 noche de alojamiento en Hotel las Dalias, categoría 3 estrellas.', 'Desayuno americano y almuerzo', 'No incluye traslados', 'No incluye tours']
['Lima', 'Explora Bandurria y Sumérgete en el Paraíso: Un tour Arqueológico y Refrescante', 'Full Day (Día completo)', '85.0', '06:30 Partida con destino la provincia de Huaura en el trayecto realizaremos una parada en una panadería artesanal para disfrutar de un delicioso desayuno. 09:00 Llegada a la paradisíaca Playa El Paraíso para disfrutar de sus tranquilas aguas. Opción de nadar en una albufera cercana co

['Madre de Dios', 'Lago Sandoval y Collpa de Loros', '03 Días / 02 Noches', '700.0', 'Día 1 Recepción en el aeropuerto o en el terminal terrestre y traslado al puerto capitanía donde tomaremos la embarcación por una hora, hasta nuestro albergue - empezaremos con una caminata introductoria. Después del almuerzo, visitaremos la Isla de los Monos. Por la noche iremos en busca de caimanes blancos, cena y descanso. Día 2 Luego del desayuno caminaremos 3 Km. en medio de la densa selva tropical y nos dirigiremos hacia el Lago Sandoval, retorno al Lodge para disfrutar del almuerzo típico. Por la tarde después de almuerzo haremos una pequeña caminata por el Canopy Walk-Way, sobre los 30 metros de altura. Por la noche tendremos una caminata donde podremos apreciar la diversidad de especies de insectos, anfibios, reptiles y otros animales de hábitos nocturnos. – cena y descanso. Día 3 Muy temprano en la mañana tomaremos el bote para visitar Collpa de Loros, una pared de tierra (arcilla) a un lado

['Piura', 'Máncora Aventurera', '03 Días / 02 Noches', '490.0', 'Día 01: Arribo a su hotel. Tarde libre. Noche de alojamiento. Día 02: Desayuno en el hotel. Traslado al muelle. Paseo Costero. Nado con Tortugas. retorno y noche de alojamiento. Día 03: Desayuno en el hotel. mañana libre. Check-Out Fin de los servicios', '02 noches de alojamiento en Hotel Máncora y Sol, categoría 2 estrellas.', '02 Desayunos en el alojamiento.', 'No incluye traslados', 'Nado con tortugas.']
['Piura', 'Vive un atardecer de manera diferente - sunset party', '02 Días / 01 Noche', '395.0', 'Dia 01: Tours Manglares Puerto Pizarro Y Sus Islas Mas Frontera Peru Ecuador Recojo del aeropuerto o ciudad y traslado Salida a nuestra excursión programada a los manglares, donde conoceremos las 4 Islas. 1. Isla de los cocodrilos. 2. Isla de los pájaros. 3. Hueso de ballena. 4. Isla del Amor/ Tiempo para almorzar en la isla del Amor y disfrutar de la playa. Opcional tienen para realizar deportes acuáticos como moto acuáti

['Puno', 'Castillo Titicaca y Templo de fertilidad', 'Full Day (Día completo)', '150.0', '09:00 hrs. Recojo de su hotel en Puno ciudad. 09:30 hrs. Llegada a CHUCUITO. 09:40 hrs. Visita del templo de la FERTILIDAD. 10:30 hrs. Visita de la plaza ( reloj solar ). 10:45 hrs. Visita del mirador. 11:00 hrs. Visita de las Caras gigante del INKA. 12:00 hrs. Llegada al Castillo del TITICACA. 12:15 hrs. Inicio de la vista sl Castillo. 13:30 hrs. Almuerzo (no incluye). 14:30 hrs. Relax en la terraza o playa, observando el majestuoso Lago TITIKAKA. 15:00 hrs. Retorno a Puno ciudad. 16:00 hrs. Llegada a Puno ciudad (hotel).', 'No incluye alojamiento', 'No incluye alimentación', 'Traslados incluidos: Puno (hotel) / Templo de la fertilidad / Castillo del TITICACA. Castillo del TITICACA/ Puno (hotel).', 'Tour Templo de la Fertilidad (Chucuito). Plaza y mirador de Chucuito. Tour Castillo del Titicaca.']
['San Martín', 'Todo lo que debes conocer cuando visites Tarapoto', '04 Días / 03 Noches', '545.0', 

['Ucayali', 'Vacaciones en Pucallpa 2024', '02 Noches', '165.0', 'No hay descripción disponible', '02 noches de alojamiento en Hotel Qallwa Pucallpa, categoría 4 estrellas', 'Incluye desayuno americano por noche de alojamiento para 2 personas..', 'Incluye recojo ida-retorno desde el aeropuerto.', 'No incluye tours']


## Desarrollo de consultas:

In [126]:
# Input para que el usuario ingrese el destino deseado
D = input("Ingresa el destino: ")

# Creando la función destino
def destino(region):
    # Para mostrar la fecha y hora de la consulta
    timestamp = time.time()
    current_date = time.ctime(timestamp)
    
    # Para realizar una conexión con la base de datos
    conn = sqlite3.connect('peru.sqlite')
    cur = conn.cursor()
    
    # Seleccionar todas las ofertas que se encuentren en la provincia deseada
    cur.execute("SELECT * FROM Actividades WHERE provincia = ?", (region,))
    x = cur.fetchall()
    
    # Mostrar la información de cada oferta
    resultados = []
    for fila in x:
        resultados.append("Provincia: {}\n"
                          "Título: {}\n"
                          "Duración: {}\n"
                          "Costo: S/ {}\n"
                          "Descripción: {}\n"
                          "Alojamiento: {}\n"
                          "Alimentación: {}\n"
                          "Traslados: {}\n"
                          "Tours: {}\n".format(fila[0], fila[1], fila[2], fila[3], fila[4], fila[5], fila[6], fila[7], fila[8]))
    
    conn.commit()
    conn.close()
    return "\nFecha y hora de consulta: {}\n\nSe han encontrado {} resultados.\n\nResultados:\n\n{}".format(current_date, len(x), '\n'.join(resultados))

# Probando la función
try:
    print(destino(D))
except:
    print("No se encontró")

Ingresa el destino: Arequipa

Fecha y hora de consulta: Sun Jan 14 22:20:06 2024

Se han encontrado 6 resultados.

Resultados:

Provincia: Arequipa
Título: Arequipa Inolvidable
Duración: 04 Días / 03 Noches
Costo: S/ 670.0
Descripción: Día 01: Traslado Apto-hotel, a hora indicada se realizará la visita a las Canteras de Sillar de Añashuayco. Noche en Arequipa. Día 02: Recojo en el hotel 07:40 a 08:10, salimos rumbo al Valle del Colca, realizando diferentes paradas, Pampa Cañahuas, mirador de los volcanes, llegada a Chivay, por la tarde visita a los baños termales Calera. Noche en Chivay. Día 03: Visita a la Cruz del cóndor y los diferentes miradores y pueblitos del Valle del Colca, retorno a la Ciudad Blanca. Noche en Arequipa. Día 04: Recojo del hotel. Partimos con dirección a la Laguna de Salinas, realizamos una parada para ver los volcanes, visitamos el salar de Huito, retornando visitaremos Chiguata. Retorno a Arequipa. Llegada aprox. 13:00 Pm. A hora indicada traslado hotel-Apto.


In [30]:
def fecha():
    timestamp = time.time()
    current_date = time.ctime(timestamp)
    return current_date

fecha()

'Sun Jan 14 21:13:07 2024'

In [22]:
# Para exportar la información en un archivo pdf
!pip install fpdf

In [143]:
# Creamos el data frame
dict = {"OPCIÓN": ID, "IDA": Duracion_ida, "VUELTA": Duracion_vuelta, "PRECIO": Precio,"AEROLÍNEA": Aerolinea} 
datfr = pd.DataFrame(dict)

# Exportamos la información en un archivo PDF
import pandas as pd
from fpdf import FPDF

# Generamos el PDF
pdf = FPDF('P', 'cm', 'A4') # Página, medidas, tamaño de página
pdf.add_page() # Agregar página

# Agregamos imagen de encabezado
pdf.image('encabezado.jpg', x = 5.5, w = 10)

# Incluímos el título del PDF
pdf.set_font('helvetica', 'B', 20)
pdf.multi_cell(0, 0.5, "Tu experiencia en: "+ D, border=False, align = 'C')
pdf.ln()

# Agregamos la fecha y hora de consulta y ciudad de origen:
pdf.set_font('helvetica', 'B', 12)
consulta = str(fecha()).strip().split(',')
for linea in consulta:
    pdf.cell(2.5, 0.5, " ", border=False)
    pdf.cell(6, 0.5, "Fecha y hora de consulta:", border=True)
    pdf.cell(3, 0.5, " ", border=False)
    pdf.multi_cell(5, 0.5, "Ciudad de origen:", border=True)
    pdf.set_font('helvetica', '', 12)
    pdf.cell(2.5, 0.5, " ", border=False)
    pdf.cell(6, 0.5, linea, border=True)
    pdf.cell(3, 0.5, " ", border=False)
    pdf.multi_cell(5, 0.5, ciudad_input, border=True)
    pdf.ln()

pdf.ln()
pdf.set_font('helvetica', 'BU', 12)
pdf.multi_cell(0, 0.5, "Resultados sobre vuelos:", border=False)
pdf.ln()

# Para el data frame de vuelos
pdf.cell(2.5, 0.5, " ", border=False)
for i in list(datfr.columns.values):
    pdf.set_font('helvetica', 'B', 10)
    pdf.cell(2.8, 0.5, str(i), border=True)
pdf.ln()
for index, row in datfr.iterrows():
    pdf.cell(2.5, 0.5, " ", border=False)
    for data in row.values:
        pdf.set_font('helvetica', '', 12)
        pdf.cell(2.8, 0.5, str(data), border=True)  # write each data for the row in its cell
    pdf.ln()  

texto = str(destino(D)).strip().split('\n')

pdf.ln()
pdf.set_font('helvetica', 'BU', 12)
pdf.multi_cell(0, 0.5, "Resultados sobre destinos:", border=False)
pdf.ln()
pdf.set_font('helvetica', '', 12)
pdf.multi_cell(0, 0.5, texto[2], border=False)

# Insertar el contenido de los destinos en el pdf
for linea in texto[5:]:
    if linea == '':
        pdf.ln(0.5)
    else:
        pdf.multi_cell(0, 0.5, linea, border=True)

# Guardamos el PDF
pdf.output("PALC_itinerario.pdf")

print("El archivo PDF se ha generado exitosamente.")

El archivo PDF se ha generado exitosamente.
